In [1]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 211 kB 13.1 MB/s 
     |████████████████████████████████| 7.4 MB 34.1 MB/s 
     |████████████████████████████████| 81 kB 1.2 MB/s 
     |████████████████████████████████| 93 kB 1.8 MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13553 sha256=f66b5c87b742629b754f55361855eac2eda91dd6f634b3b54b8c4a8d2519b3c8
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3357 sha256=bdb479ac7d6973978266862a8e15a0aacc368885813af88769e0a9348b6a9e43
  Stored in directory: /root/.cache/pip/wheels/7f/d4/8f/6e2ca54744c9d7292d88ddb8d42876bcdab5e6d84a21c10346
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398404 sha256=4c0682a667a28f9bd7cf6e61da795f10a4e6a759afb55b74

In [3]:
#import the packages
from newspaper import Article
import random
import string
import numpy as np
import warnings
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
#ignore the warnings
warnings.filterwarnings('ignore')

In [5]:
#download package from nltk
nltk.download('punkt',quiet=True)
nltk.download('wordnet',quiet=True)

True

In [6]:
#get artical url
# article= Article('https://simple.wikipedia.org/wiki/Light')
# article= Article('https://www.cdc.gov/heartdisease/journal_articles.htm')
# article= Article('https://www.mayoclinic.org/diseases-conditions/parkinsons-disease/symptoms-causes/syc-20376055')
article= Article('https://en.wikipedia.org/wiki/Health_care')    
article.download()
article.parse()
article.nlp()
corpus=article.text
#print
print(corpus)

Prevention of disease and promotion of wellbeing

[ needs update ] Global concentrations of health care resources, as depicted by the number of physicians per 10,000 individuals, by country. Data is sourced from a World Health Statistics 2010, a WHO report.

Health care or healthcare is the improvement of health via the prevention, diagnosis, treatment, amelioration, or cure of disease, illness, injury, and other physical and mental impairments in people. Health care is delivered by health professionals and allied health fields. Medicine, dentistry, pharmacy, midwifery, nursing, optometry, audiology, psychology, occupational therapy, physical therapy, athletic training, and other health professions are all part of health care. It includes work done in providing primary care, secondary care, and tertiary care, as well as in public health.

Access to health care may vary across countries, communities, and individuals, influenced by social and economic conditions as well as health policie

In [7]:
#tokenization
text=corpus
sent_tokens=nltk.sent_tokenize(text)
print(sent_tokens)

['Prevention of disease and promotion of wellbeing\n\n[ needs update ] Global concentrations of health care resources, as depicted by the number of physicians per 10,000 individuals, by country.', 'Data is sourced from a World Health Statistics 2010, a WHO report.', 'Health care or healthcare is the improvement of health via the prevention, diagnosis, treatment, amelioration, or cure of disease, illness, injury, and other physical and mental impairments in people.', 'Health care is delivered by health professionals and allied health fields.', 'Medicine, dentistry, pharmacy, midwifery, nursing, optometry, audiology, psychology, occupational therapy, physical therapy, athletic training, and other health professions are all part of health care.', 'It includes work done in providing primary care, secondary care, and tertiary care, as well as in public health.', 'Access to health care may vary across countries, communities, and individuals, influenced by social and economic conditions as we

In [8]:
#creating a dictionary to remove the punctuation
remove_punct_dict=dict( (ord(punct),None) for punct in string.punctuation)
print(string.punctuation)
print(remove_punct_dict)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [9]:
#create a function to return lower case words 
def LemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))
print(LemNormalize(text))

['prevention', 'of', 'disease', 'and', 'promotion', 'of', 'wellbeing', 'needs', 'update', 'global', 'concentrations', 'of', 'health', 'care', 'resources', 'as', 'depicted', 'by', 'the', 'number', 'of', 'physicians', 'per', '10000', 'individuals', 'by', 'country', 'data', 'is', 'sourced', 'from', 'a', 'world', 'health', 'statistics', '2010', 'a', 'who', 'report', 'health', 'care', 'or', 'healthcare', 'is', 'the', 'improvement', 'of', 'health', 'via', 'the', 'prevention', 'diagnosis', 'treatment', 'amelioration', 'or', 'cure', 'of', 'disease', 'illness', 'injury', 'and', 'other', 'physical', 'and', 'mental', 'impairments', 'in', 'people', 'health', 'care', 'is', 'delivered', 'by', 'health', 'professionals', 'and', 'allied', 'health', 'fields', 'medicine', 'dentistry', 'pharmacy', 'midwifery', 'nursing', 'optometry', 'audiology', 'psychology', 'occupational', 'therapy', 'physical', 'therapy', 'athletic', 'training', 'and', 'other', 'health', 'professions', 'are', 'all', 'part', 'of', 'hea

In [10]:
#keywords for greetings
greeting_input=["hi","hello","hey","hola"]
greeting_response=["howdy","hey there","hi","hello :)"]
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in greeting_input:
      return random.choice(greeting_response)

In [11]:
def response(user_response):
 #user response and robo responce
  #user_response="WHat is chronic disease"
  user_response=user_response.lower()
  #print(user_response)
  #robo response
  robo_response=''
  sent_tokens.append(user_response)
  #print(sent_tokens)
  tfidfvec=TfidfVectorizer(tokenizer=LemNormalize , stop_words='english')
  tfidf=tfidfvec.fit_transform(sent_tokens)
  #print(tfidf)
  #get similarity score
  val=cosine_similarity(tfidf[-1],tfidf)
  #print(val)
  idx=val.argsort()[0][-2]
  flat=val.flatten()
  flat.sort()
  score=flat[-2]
  #print(score)
  if score==0:
    robo_response=robo_response+"sorry,i dont understand"
  else:
    robo_response=robo_response+sent_tokens[idx]

  sent_tokens.remove(user_response)
  return robo_response

In [12]:
greeting_input=["hi","hello","hey","hola"]
greeting_response=["howdy","hey there","hi","hello :)"]
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in greeting_input:
      return random.choice(greeting_response)
flag=True
print("hello!!! this is self_Chatbot,i can answer your queris related to HealthCare ,type bye to exit")
while(flag==True):
  user_response=input("me:")
  #user_response=user_response.lower()
  if(user_response!='bye'):
    if(user_response=='thanks' or user_response=='thank you'):
      flag=False
      print("self_Chatbot: anytime :)")
    else:
       if( greeting(user_response) != None):
         print("self_Chatbot: "+ greeting(user_response))
       else:
         print("self_Chatbot:"+response(user_response))
  else:
    flag=False
    print("self_Chatbot: see you later :)")

hello!!! this is self_Chatbot,i can answer your queris related to HealthCare ,type bye to exit
me:hi
self_Chatbot: hello :)
me:hii
self_Chatbot:sorry,i dont understand
me:Healthcare
self_Chatbot:Healthcare industry [ edit ]

The healthcare industry incorporates several sectors that are dedicated to providing health care services and products.
me:sun
self_Chatbot:sorry,i dont understand
me:service
self_Chatbot:Physicians in this model bill patients directly for services, either on a pre-paid monthly, quarterly, or annual basis, or bill for each service in the office.
me:Drugs
self_Chatbot:sorry,i dont understand
me:drugs
self_Chatbot:sorry,i dont understand
me:sectors
self_Chatbot:This evaluation of quality is based on measures of:

hospital quality

health plan quality

physician quality

quality for other health professionals

of patient experience

Related sectors [ edit ]

Health care extends beyond the delivery of services to patients, encompassing many related sectors, and is set 